In [0]:
# Cell 0: Imports
import os
import uuid
import json
import base64
import pandas as pd
from io import BytesIO
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
from delta.tables import DeltaTable

from clinical_data_standards_framework.utils import (
    save_with_audit,
    find_excel_header_row,
    get_header_filter_patterns,
    parse_max_length,
    is_required_field,
    is_transfer_key_field,
    get_transfer_metadata_flexible_mapper,
    compute_definition_hash  # For inline hash computation
)

In [ ]:
# Cell 1: Check if Previous Task Found Documents
# Skip this task if extract_excel_sheets found no documents

try:
    documents_found = dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="documents_found")
    
    if documents_found == "false":
        print("⏭️  No documents were found in previous task (extract_excel_sheets)")
        print("   Skipping transfer metadata processing.")
        dbutils.notebook.exit("Skipped - no documents to process")
    
    print(f"✓ Documents found in previous task: {documents_found}")
    
except Exception as e:
    # If task value doesn't exist, assume we should continue (backwards compatibility)
    print(f"⚠ Could not read 'documents_found' task value: {e}")
    print("  Continuing anyway (backwards compatibility)")
    pass


In [0]:
# Cell 1: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
bronze_schema = globals_dict['bronze_schema']
silver_schema = globals_dict['silver_schema']

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Silver Schema: {silver_schema}")
print(f"Created by: {created_by_principal}")
print(f"Job ID: {databricks_job_id}, Job Name: {databricks_job_name}, Run ID: {databricks_run_id}")

In [ ]:
# Cell 3: Read Validation Configuration
# Get required columns from pipeline config
validation_config = pipeline_config.get('validation', {}).get('transfer_metadata', {})
required_columns_final = validation_config.get('required_columns', [
    # Default required columns if not in config (using final schema field names)
    'transfer_variable_name',
    'transfer_variable_order', 
    'format',
    'anticipated_max_length',
    'transfer_file_key',
    'populate_for_all_records'
])

print(f"Required columns (from config): {required_columns_final}")

# Map final schema field names to raw Excel column names
# This mapping is needed because Excel has _raw suffix for fields that need transformation
field_to_raw_map = {
    'transfer_variable_name': 'transfer_variable_name',
    'transfer_variable_order': 'transfer_file_order',  # Different name in Excel
    'format': 'data_type_format',  # Different name in Excel
    'anticipated_max_length': 'anticipated_max_length_raw',  # Needs transformation
    'transfer_file_key': 'transfer_file_key',
    'populate_for_all_records': 'populate_for_all_records_raw'  # Needs transformation
}

# Convert required columns to raw column names for Excel validation
required_columns_raw = [field_to_raw_map.get(col, col) for col in required_columns_final]
print(f"Required columns (raw Excel names): {required_columns_raw}")


In [0]:
# Cell 2: Construct Table Names
sheets_table = f"{catalog}.{bronze_schema}.md_dta_excel_file_raw"
history_table = f"{catalog}.{bronze_schema}.md_dta_history"
status_table = f"{catalog}.{bronze_schema}.md_document_status"
codelist_table = f"{catalog}.silver_md.md_codelists_normalized"  # Lookup table for codelist values
output_table = f"{catalog}.silver_md.md_transfer_variable_field_normalized"  # Target normalized silver table

print(f"Sheets table: {sheets_table}")
print(f"History table: {history_table}")
print(f"Status table: {status_table}")
print(f"Codelist table: {codelist_table}")
print(f"Output table: {output_table}")

In [0]:
# Cell 3: Read Transfer Metadata Sheets with Parent Content
# Join sheet metadata with parent document content (base64)
# Filter out already successfully processed sheets (row_status = COMPLETED)

print(f"{'='*80}")
print(f"Reading transfer_metadata sheets...")
print(f"{'='*80}")

# Read sheet metadata filtered for transfer_metadata category
# Include trial_id, data_stream_type, data_provider_name for composite PK
df_sheets = spark.table(sheets_table).filter(
    F.col("sheet_category") == "transfer_metadata"
).select(
    "document_id",
    "parent_document_id",
    "sheet_name",
    "source_file_path",
    "trial_id",
    "data_stream_type",
    "data_provider_name"
)

total_sheets = df_sheets.count()
print(f"Total transfer_metadata sheets in source: {total_sheets}")

# Check which parent documents have already been processed
# NOTE: We track by parent_document_id to allow reprocessing new versions of the same trial/stream/provider
if spark.catalog.tableExists(output_table):
    # Get list of parent_document_id values that have already been processed
    # We use parent_document_id because each file version gets a unique ID
    df_already_processed = spark.table(output_table).select(
        F.col("parent_document_id")
    ).distinct()
    
    # Anti-join to exclude sheets from already processed parent documents
    # This allows new versions (different parent_document_id) to be processed
    df_sheets_to_process = df_sheets.join(
        df_already_processed,
        on="parent_document_id",
        how="left_anti"
    )
    
    already_processed_count = total_sheets - df_sheets_to_process.count()
    print(f"Sheets from already processed documents: {already_processed_count}")
    print(f"Sheets from new documents to process: {df_sheets_to_process.count()}")
else:
    print(f"Output table does not exist yet - will create it")
    df_sheets_to_process = df_sheets

# Join with parent documents to get content_base64
# Only join with current versions (defensive filter - sheets should already be from current versions)
df_with_content = df_sheets_to_process.join(
    spark.table(history_table).filter(
        F.col("is_current") == True  # Only current versions
    ).select(
        F.col("document_id").alias("parent_id"),
        "content_base64"
    ),
    df_sheets_to_process.parent_document_id == F.col("parent_id")
)

sheet_count = df_with_content.count()
print(f"Sheets to process in this run: {sheet_count}")
print(f"{'='*80}")

if sheet_count == 0:
    print("✓ No new transfer_metadata sheets to process. All up-to-date.")
    # Set task values for downstream tasks
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value="UP_TO_DATE")
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value="0")
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value="0")
    dbutils.notebook.exit("All transfer_metadata sheets already processed")

In [0]:
# Cell 4: Define Output Schema
# Schema for normalized transfer variable fields table
# Composite PK: (trial_id, data_stream_type, data_provider_name, transfer_variable_name)

output_schema = StructType([
    StructField("transfer_variable_id", StringType(), False),  # PK - UUID
    StructField("parent_document_id", StringType(), False),  # FK to md_dta_history (for version tracking)
    StructField("trial_id", StringType(), False),
    StructField("data_stream_type", StringType(), False),
    StructField("data_provider_name", StringType(), False),
    StructField("transfer_variable_name", StringType(), False),
    StructField("codelist_values", ArrayType(StringType()), True),  # Populated by codelist task
    StructField("transfer_variable_label", StringType(), True),
    StructField("variable_description", StringType(), True),
    StructField("transfer_variable_order", IntegerType(), True),
    StructField("format", StringType(), True),
    StructField("anticipated_max_length", IntegerType(), True),  # Parsed from _raw
    StructField("populate_for_all_records", BooleanType(), True),
    StructField("transfer_file_key", BooleanType(), True),
    StructField("example_values", StringType(), True),
    StructField("definition_hash", StringType(), True),  # Populated by hash generator task
    StructField("categorization_notes", StringType(), True),
    StructField("row_status", StringType(), True)
])

print(f"Output schema defined with {len(output_schema.fields)} fields")
print(f"Primary Key: transfer_variable_id (UUID)")
print(f"Natural Key: (trial_id, data_stream_type, data_provider_name, transfer_variable_name)")


In [0]:
# Cell 5: Process Transfer Metadata Sheets
# Parse each sheet and extract transfer variable fields
# NOTE: Codelists are processed FIRST and we look them up here

print(f"{'='*80}")
print(f"Processing transfer metadata sheets...")
print(f"{'='*80}")

# Load codelist lookup dictionary (from codelist table processed earlier)
# Key: (trial_id, data_stream_type, data_provider_name, transfer_variable_name)
# Value: list of codelist values
codelist_lookup = {}
try:
    if spark.catalog.tableExists(codelist_table):
        codelist_df = spark.table(codelist_table).select(
            "trial_id", "data_stream_type", "data_provider_name", 
            "transfer_variable_name", "values"
        ).collect()
        
        for cl_row in codelist_df:
            key = (
                cl_row['trial_id'], 
                cl_row['data_stream_type'], 
                cl_row['data_provider_name'], 
                cl_row['transfer_variable_name']
            )
            codelist_lookup[key] = cl_row['values'] if cl_row['values'] else []
        
        print(f"Loaded {len(codelist_lookup)} codelist entries for lookup")
    else:
        print(f"⚠ Codelist table not found: {codelist_table}")
        print("  Will proceed without codelist values")
except Exception as e:
    print(f"⚠ Error loading codelists: {e}")
    print("  Will proceed without codelist values")

# Hash fields for definition_hash computation
hash_fields = ['transfer_variable_name', 'transfer_variable_order', 'format', 
               'anticipated_max_length', 'transfer_file_key', 'populate_for_all_records', 
               'codelist_values_str']

all_records = []

for row in df_with_content.collect():
    sheet_name = row['sheet_name']
    source_path = row['source_file_path']
    content_b64 = row['content_base64']
    parent_document_id = row['parent_document_id']  # For version tracking
    
    # Metadata for natural key
    trial_id = row['trial_id']
    data_stream_type = row['data_stream_type']
    data_provider_name = row['data_provider_name']
    
    print(f"\nProcessing: {sheet_name}")
    print(f"  Trial: {trial_id}, Stream: {data_stream_type}, Provider: {data_provider_name}")
    print(f"  Parent Document ID: {parent_document_id}")
    
    try:
        # Decode parent Excel file from base64
        excel_bytes = base64.b64decode(content_b64)
        
        # Detect engine based on file extension
        file_ext = os.path.splitext(source_path.lower())[1]
        engine = 'xlrd' if file_ext in ['.xls', '.xlt'] else 'openpyxl'
        
        # Read sheet to find header row
        raw_df = pd.read_excel(BytesIO(excel_bytes), sheet_name=sheet_name, header=None, engine=engine)
        header_row = find_excel_header_row(raw_df, [
            r'transfer.*variable.*name',
            r'format'
        ])
        
        # Read sheet with proper header
        df_sheet = pd.read_excel(
            BytesIO(excel_bytes),
            sheet_name=sheet_name,
            header=header_row,
            engine=engine
        ).dropna(axis=1, how='all')
        
        print(f"  Total columns in Excel: {len(df_sheet.columns)}")
        print(f"  Original columns:")
        for i, col in enumerate(df_sheet.columns):
            # Show FULL column name, not truncated
            print(f"    [{i}] '{col}'")
        
        # Use flexible column mapper with configurable required columns
        col_map, missing_required = get_transfer_metadata_flexible_mapper(
            list(df_sheet.columns), 
            required_cols_raw=required_columns_raw
        )
        
        print(f"\n  Mapped {len(col_map)} columns:")
        for excel_col, std_name in col_map.items():
            # Show FULL column name, not truncated
            print(f"    '{excel_col}' → '{std_name}'")
        
        # Show unmapped columns
        unmapped = [col for col in df_sheet.columns if col not in col_map]
        if unmapped:
            print(f"\n  ⚠ Unmapped columns ({len(unmapped)}):")
            for col in unmapped:
                # Show FULL column name, not truncated
                print(f"    - '{col}'")
        
        # Check for missing required columns
        sheet_status = None
        sheet_notes = None
        if missing_required:
            sheet_status = "MANUAL_REVIEW_REQUIRED"
            sheet_notes = f"Missing required columns: {', '.join(missing_required)}"
            print(f"  ⚠ WARNING: {sheet_notes}")
        
        # Rename columns using flexible mapper
        df_sheet = df_sheet.rename(columns=col_map)
        
        # Filter to rows with transfer_variable_name
        if "transfer_variable_name" not in df_sheet.columns:
            print(f"  ⚠ WARNING: No transfer_variable_name column found")
            print(f"  → This sheet will be marked for manual review but processing continues")
            # Don't skip - mark sheet status and continue processing
            if not sheet_status:
                sheet_status = "MANUAL_REVIEW_REQUIRED"
            if not sheet_notes:
                sheet_notes = "Critical column 'transfer_variable_name' not found"
            else:
                sheet_notes += "; Critical column 'transfer_variable_name' not found"
            # Create empty dataframe to avoid errors below
            df_sheet = pd.DataFrame()
        else:
            df_sheet = df_sheet[df_sheet["transfer_variable_name"].notna()].copy()
        
        # Debug: Show first 3 rows before header filtering
        print(f"  First 3 rows before header filtering:")
        if len(df_sheet) > 0:
            for idx in range(min(3, len(df_sheet))):
                row_data = df_sheet.iloc[idx]
                var_name = row_data.get('transfer_variable_name', 'N/A')
                file_order = row_data.get('transfer_file_order', 'N/A')
                print(f"    Row {idx}: transfer_variable_name='{var_name}', transfer_file_order='{file_order}'")
        
        # Get header patterns for transfer_metadata sheets
        header_patterns = get_header_filter_patterns('transfer_metadata')
        
        # Check each column for header text and create a combined mask
        header_mask = pd.Series([False] * len(df_sheet), index=df_sheet.index)
        
        for col, patterns in header_patterns.items():
            if col in df_sheet.columns:
                for pattern in patterns:
                    mask = df_sheet[col].astype(str).str.lower().str.contains(
                        pattern, na=False, regex=True
                    )
                    if mask.any():
                        print(f"    Found header pattern '{pattern}' in column '{col}': {mask.sum()} rows")
                    header_mask = header_mask | mask
        
        # Remove header rows
        if header_mask.any():
            header_row_count = header_mask.sum()
            print(f"    ⚠ Removing {header_row_count} header-like row(s) from data")
            print(f"    Original row count: {len(df_sheet)}")
            df_sheet = df_sheet[~header_mask].copy()
            print(f"    Remaining rows: {len(df_sheet)}")
        else:
            print(f"    ✓ No header rows detected in data")
        
        print(f"  Found {len(df_sheet)} transfer variables")
        
        # If no rows and sheet has issues, log it
        if len(df_sheet) == 0 and sheet_status:
            print(f"  ⚠ Sheet produced 0 records due to: {sheet_notes}")
            print(f"  → Sheet will be skipped but task continues")
        
        # Debug: Check if key columns exist after rename and filtering
        print(f"\n  Columns after rename and filtering ({len(df_sheet.columns)} total):")
        print(f"    {list(df_sheet.columns)}")
        
        # Check for specific columns we care about
        key_columns = ['transfer_file_key', 'variable_description', 'populate_for_all_records_raw', 'anticipated_max_length_raw']
        for col in key_columns:
            if col in df_sheet.columns:
                non_null_count = df_sheet[col].notna().sum()
                print(f"    ✓ '{col}' exists: {non_null_count}/{len(df_sheet)} non-null values")
                # Show first 5 values
                sample_vals = df_sheet[col].head(5).tolist()
                print(f"      Sample values: {sample_vals}")
            else:
                print(f"    ✗ '{col}' MISSING from DataFrame!")
        
        # Parse each variable row
        for idx, r in df_sheet.iterrows():
            # Debug: Show first row values for key columns
            if idx == df_sheet.index[0]:  # First row only
                print(f"\n  Debug - First row values:")
                print(f"    transfer_variable_name: '{r.get('transfer_variable_name')}'")
                print(f"    transfer_file_order: '{r.get('transfer_file_order')}'")
                print(f"    transfer_file_key: '{r.get('transfer_file_key')}' (raw)")
                print(f"    variable_description: '{r.get('variable_description')}' (raw)")
                print(f"    populate_for_all_records_raw: '{r.get('populate_for_all_records_raw')}'")
                print(f"    Available columns in row: {list(r.index)[:10]}...")
            
            # Determine row-level status
            row_status = sheet_status  # Inherit sheet-level issues
            row_notes = sheet_notes
            
            # Check for missing required values in this row (using configurable required columns)
            row_missing = []
            for req_col in required_columns_raw:
                if req_col in df_sheet.columns:
                    val = r.get(req_col)
                    if pd.isna(val) or (isinstance(val, str) and str(val).strip() == ''):
                        row_missing.append(req_col)
                else:
                    # Column not in sheet (already flagged at sheet level)
                    pass
            
            if row_missing and not row_status:
                row_status = "MANUAL_REVIEW_REQUIRED"
                row_notes = f"Missing required values: {', '.join(row_missing)}"
            
            # If no issues found, mark as COMPLETED
            if not row_status:
                row_status = "COMPLETED"
            
            record = {
                # Primary Key (UUID)
                'transfer_variable_id': str(uuid.uuid4()),
                
                # Foreign Key (for version tracking)
                'parent_document_id': parent_document_id,
                
                # Natural Key
                'trial_id': trial_id,
                'data_stream_type': data_stream_type,
                'data_provider_name': data_provider_name,
                'transfer_variable_name': str(r.get('transfer_variable_name', '')),
                
                # Codelist values (looked up from codelist table)
                'codelist_values': codelist_lookup.get(
                    (trial_id, data_stream_type, data_provider_name, str(r.get('transfer_variable_name', ''))),
                    None
                ),
                
                # Variable attributes
                'transfer_variable_label': str(r.get('transfer_variable_label')) if pd.notna(r.get('transfer_variable_label')) else None,
                'variable_description': str(r.get('variable_description')) if pd.notna(r.get('variable_description')) else None,
                'transfer_variable_order': int(float(r.get('transfer_file_order'))) if pd.notna(r.get('transfer_file_order')) and str(r.get('transfer_file_order')).replace('.','',1).replace('-','',1).isdigit() else None,
                'format': str(r.get('data_type_format')) if pd.notna(r.get('data_type_format')) else None,
                'anticipated_max_length': parse_max_length(r.get('anticipated_max_length_raw'))[0],  # Extract int from tuple
                
                # Boolean flags (handles pandas NaN properly)
                'populate_for_all_records': is_required_field(r.get('populate_for_all_records_raw')),
                'transfer_file_key': is_transfer_key_field(r.get('transfer_file_key')),
                
                # Optional fields
                'example_values': str(r.get('example_values')) if pd.notna(r.get('example_values')) else None,
                'definition_hash': None,  # Will be computed below
                
                # Error tracking
                'categorization_notes': row_notes,
                'row_status': row_status
            }
            
            # Compute definition_hash inline (for append-only silver table)
            # Convert codelist_values to sorted, pipe-delimited string for hashing
            codelist_vals = record.get('codelist_values')
            if codelist_vals:
                codelist_str = '|'.join(sorted([str(v).lower() for v in codelist_vals]))
            else:
                codelist_str = ''
            
            # Build hash record with fields used for deduplication
            hash_record = {
                'transfer_variable_name': record.get('transfer_variable_name') or '',
                'transfer_variable_order': str(record.get('transfer_variable_order') or ''),
                'format': record.get('format') or '',
                'anticipated_max_length': str(record.get('anticipated_max_length') or ''),
                'transfer_file_key': str(record.get('transfer_file_key') or ''),
                'populate_for_all_records': str(record.get('populate_for_all_records') or ''),
                'codelist_values_str': codelist_str
            }
            
            # Compute definition hash
            record['definition_hash'] = compute_definition_hash(hash_record, list(hash_record.keys()))
            
            all_records.append(record)
        
        print(f"  ✓ Processed {len(df_sheet)} variables")
        
    except Exception as e:
        print(f"  ✗ ERROR processing sheet: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*80}")
print(f"✓ Processing complete: {len(all_records)} total records")
print(f"{'='*80}")


In [0]:
# Cell 6: Write Results to Silver Table & Set Task Values
# Create DataFrame, write with audit columns, and signal success/failure to downstream tasks

if len(all_records) == 0:
    print("No records to write. Exiting.")
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value="NO_DATA")
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value="0")
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value="0")
    dbutils.notebook.exit("No transfer variables found to process")

# Count success vs error records
success_count = len([r for r in all_records if r.get('row_status') == 'COMPLETED'])
error_count = len([r for r in all_records if r.get('row_status') and r['row_status'] != 'COMPLETED'])

print(f"\n{'='*80}")
print(f"Processing Summary:")
print(f"  Total records: {len(all_records)}")
print(f"  Success (COMPLETED): {success_count}")
print(f"  Errors/Manual Review: {error_count}")
print(f"{'='*80}")

# Create Spark DataFrame
df_result = spark.createDataFrame(all_records, schema=output_schema)

# Display sample
print(f"\nSample records:")
df_result.show(5, truncate=False)

# Write to silver table with audit columns using APPEND mode for full lineage
# Deduplication happens at the gold layer using definition_hash
print(f"\nWriting {len(all_records)} records to {output_table}...")
save_with_audit(
    df=df_result,
    table_name=output_table,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    mode='append',  # Use append for full lineage (dedup at gold layer)
    table_description="The table contains information about transfer variables used in trials. It includes details such as the variable ID, trial ID, data stream type, and provider name. This data can be utilized for tracking and analyzing the characteristics of transfer variables, understanding their usage in different trials, and ensuring consistency in data formats. Use cases may include data validation, reporting on variable definitions, and managing variable attributes across various trials."
)

print(f"\n{'='*80}")
print(f"✓ Successfully wrote {len(all_records)} records")
print(f"  Table: {output_table}")
print(f"  Mode: append (full lineage, dedup at gold layer)")
print(f"  Audit columns added automatically")
print(f"{'='*80}")

# Set task values for conditional downstream execution
if success_count == 0 and error_count > 0:
    # All records need manual review - but DON'T fail the task
    task_status = "MANUAL_REVIEW_REQUIRED"
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value=task_status)
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value=str(success_count))
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value=str(error_count))
    print(f"\n⚠ WARNING: All {error_count} transfer variable records need manual review")
    print(f"  Check categorization_notes and row_status columns for details")
    print(f"  Task will complete successfully - data has been written for review")
elif success_count > 0:
    # At least some success - downstream can proceed
    task_status = "SUCCESS"
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value=task_status)
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value=str(success_count))
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value=str(error_count))
    print(f"\n✓ Task status: {task_status}")
    print(f"  Downstream tasks will proceed.")
else:
    # No records at all (shouldn't reach here due to earlier exit, but handle it)
    task_status = "NO_DATA"
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value=task_status)
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value="0")
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value="0")

In [0]:
# Cell 7: Verify Results
# Query the table to verify the write

print(f"\nVerifying table: {output_table}")
result_count = spark.table(output_table).count()
print(f"Total records in table: {result_count}")

# Show sample
print(f"\nSample records from {output_table}:")
spark.table(output_table).select(
    "transfer_variable_id",
    "trial_id",
    "transfer_variable_name",
    "transfer_variable_order",
    "format",
    "anticipated_max_length",
    "populate_for_all_records",
    "transfer_file_key",
    "example_values",
    "row_status",
    "categorization_notes",
    "databricks_job_name"
).orderBy("trial_id", "transfer_variable_order").show(10, truncate=False)